In [1]:
import gpxpy
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from ipywidgets import widgets, interact
from geopy.distance import geodesic
import numpy as np
from ipywidgets import widgets, interact
import folium
from folium.plugins import TimestampedGeoJson
from folium.features import DivIcon, FeatureGroup
from folium import IFrame
import json
import os
import base64
from io import BytesIO
import plotly.graph_objects as go
from branca.element import Figure
import plotly.io as pio
from scipy.signal import savgol_filter
from plotly.subplots import make_subplots


In [2]:
def load_gpx_files(directory):
    gpx_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.gpx')]
    print(f"Found {len(gpx_files)} GPX files.")
    return [(extract_gpx_data(f), os.path.basename(f)) for f in gpx_files]



# Function to extract GPX data
def extract_gpx_data(gpx_file_path):
    with open(gpx_file_path, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    data = []
    for track in gpx.tracks:
        for segment in track.segments:
            previous_point = None
            for point in segment.points:
                if previous_point is not None:
                    time_diff = (point.time - previous_point.time).total_seconds()
                    distance = geodesic((previous_point.latitude, previous_point.longitude), (point.latitude, point.longitude)).meters
                    speed_mps = distance / time_diff if time_diff > 0 else 0
                    speed_kts = speed_mps * 1.94384  # Convert m/s to knots
                    heading = calculate_heading(previous_point.latitude, previous_point.longitude, point.latitude, point.longitude)
                    data.append({
                        "time": pd.to_datetime(point.time).tz_convert('UTC'),
                        "latitude": point.latitude,
                        "longitude": point.longitude,
                        "speed_kts": speed_kts,
                        "heading": heading,
                        "distance": distance
                    })
                else:
                    data.append({
                        "time": pd.to_datetime(point.time).tz_convert('UTC'),
                        "latitude": point.latitude,
                        "longitude": point.longitude,
                        "speed_kts": 0,
                        "heading": 0,
                        "distance": 0
                    })
                previous_point = point
    return pd.DataFrame(data)

# Function to calculate heading between two points
def calculate_heading(lat1, lon1, lat2, lon2):
    dLon = np.radians(lon2 - lon1)
    y = np.sin(dLon) * np.cos(np.radians(lat2))
    x = np.cos(np.radians(lat1)) * np.sin(np.radians(lat2)) - np.sin(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.cos(dLon)
    return (np.degrees(np.arctan2(y, x)) + 360) % 360

# Function to calculate VMG to a specific mark
def calculate_vmg(point, mark_lat, mark_lon):
    boat_speed = point['speed_kts']
    bearing_to_mark = calculate_heading(point['latitude'], point['longitude'], mark_lat, mark_lon)
    angle_to_mark = (bearing_to_mark - point['heading'] + 360) % 360
    return boat_speed * np.cos(np.radians(angle_to_mark))  # Cosine of the angle to the mark


# Function to select the best VMG
def select_vmg(row):
    vmg_windward = row['VMG_Windward']
    vmg_leeward = row['VMG_Leeward']
    vmg_reach = row.get('VMG_Reach', float('-inf'))  # Default to -inf if 'VMG_Reach' is not present

    # Case 1: If moving away from leeward mark, prefer windward VMG
    if vmg_leeward < 0:
        return vmg_windward

    # Case 2: Potential reach leg (either to gybe mark or from gybe to leeward)
    if vmg_reach > 0 or (vmg_reach < 0 and vmg_leeward > 0):
        # If reach VMG is positive or if we're potentially on the leg from gybe to leeward
        return max(vmg_reach, vmg_leeward)

    # Case 3: Standard upwind/downwind legs
    if vmg_windward > vmg_leeward:
        return vmg_windward
    else:
        return vmg_leeward


def update_vmg(df, marks, time_start):
    # Convert start_time to pandas Timestamp if it's not already
    time_start = pd.to_datetime(time_start)

    # Filter the DataFrame to include only rows after start_time
    df_race = df[df['time'] >= time_start].copy()

    for mark_name, coords in marks.items():
        if coords[0] is not None and coords[1] is not None:  # Ensure the mark coordinates are valid
            mark_lat, mark_lon = coords  # Unpack the coordinates
            
            # Calculate VMG only for rows after start_time
            df_race[f'VMG_{mark_name}'] = df_race.apply(
                lambda row: calculate_vmg(row, mark_lat, mark_lon),
                axis=1
            )

    # Apply select_vmg only for rows after start_time
    df_race['VMG'] = df_race.apply(select_vmg, axis=1)

    # Determine Sailing Mode
    df_race['Sailing_Mode'] = df_race.apply(determine_sailing_mode, axis=1)

    # Merge the calculated VMGs and Sailing Mode back into the original DataFrame
    columns_to_merge = ['time'] + [f'VMG_{mark}' for mark in marks.keys()] + ['VMG', 'Sailing_Mode']
    df = df.merge(df_race[columns_to_merge], on='time', how='left')

    # Fill NaN values with 0 for rows before start_time
    vmg_columns = [f'VMG_{mark}' for mark in marks.keys()] + ['VMG']
    df[vmg_columns] = df[vmg_columns].fillna(0)
    df['Sailing_Mode'] = df['Sailing_Mode'].fillna('Not Racing')

    return df  # Return the updated DataFrame

def determine_sailing_mode(row):
    vmg_windward = row['VMG_Windward']
    vmg_leeward = row['VMG_Leeward']
    vmg_reach = row.get('VMG_Reach', 0)  # Use 0 if VMG_Reach is not present
    
    if vmg_windward > max(vmg_leeward, vmg_reach, 0):
        return 'Upwind'
    elif vmg_leeward > max(vmg_windward, vmg_reach, 0):
        return 'Downwind'
    elif vmg_reach > max(vmg_windward, vmg_leeward, 0):
        return 'Reach'
    else:
        return 'Other'  # For cases where all VMGs are negative or zero

def process_all_data(data_list, marks, time_start):
    updated_data = []
    for df, file_name in data_list:
        print(f"Processing file: {file_name}")
        updated_df = update_vmg(df, marks, time_start)
        updated_data.append((updated_df, file_name))
        print(f"Processed {file_name}, DataFrame shape: {updated_df.shape}")
    return updated_data

In [3]:
# Define the directory containing the .gpx files using an environment variable
directory_path = os.getenv('GPX_DIRECTORY')

# Call the function to load all .gpx files from the directory
if directory_path:
    all_gpx_data_with_names = load_gpx_files(directory_path)
else:
    print("Environment variable GPX_DIRECTORY not set.")

Found 2 GPX files.


In [4]:
def plot_combined_vmg_and_efficiency(all_gpx_data_with_names, time_start, time_finish):
    fig = make_subplots(rows=1, cols=2, subplot_titles=("VMG Over Time", "Upwind Efficiency Comparison"),
                        shared_xaxes=True, horizontal_spacing=0.02)
    competitor_vmg_avg = {}
    competitor_efficiencies = {}
    colors = ['blue', 'red', 'green', 'purple', 'orange', 'cyan']

    for i, (df, file_name) in enumerate(all_gpx_data_with_names):
        competitor_id = file_name[:6]
        color = colors[i % len(colors)]
        
        # Filter data
        mask = (df['time'] >= time_start) & (df['time'] <= time_finish)
        df_filtered = df.loc[mask].copy()
        
        # VMG Plot
        df_filtered_vmg = df_filtered[df_filtered['VMG'] >= 0].copy()
        df_filtered_vmg.set_index('time', inplace=True)
        df_filtered_vmg['VMG_smooth'] = df_filtered_vmg['VMG'].rolling(window='60s', center=True, min_periods=1).mean()  # Increased window size
        df_filtered_vmg.reset_index(inplace=True)
        
        fig.add_trace(go.Scatter(x=df_filtered_vmg['time'], y=df_filtered_vmg['VMG_smooth'], 
                                 mode='lines', name=f"{competitor_id}", 
                                 line=dict(color=color, width=2),
                                 legendgroup=competitor_id), row=1, col=1)
        
        avg_vmg = df_filtered_vmg['VMG_smooth'].mean()
        competitor_vmg_avg[competitor_id] = avg_vmg

        # Upwind Efficiency Plot
        df_filtered.loc[df_filtered['Sailing_Mode'] != 'Upwind', 'VMG_Windward'] = np.nan
        df_filtered['upwind_efficiency'] = np.where(
            df_filtered['Sailing_Mode'] == 'Upwind',
            df_filtered['VMG_Windward'] / df_filtered['speed_kts'] * 100,
            np.nan
        )
        
        df_filtered.set_index('time', inplace=True)
        df_filtered['upwind_efficiency_smooth'] = df_filtered['upwind_efficiency'].rolling(
            window='60s', center=True, min_periods=1
        ).mean()  # Increased window size
        df_filtered.reset_index(inplace=True)
        
        fig.add_trace(go.Scatter(x=df_filtered['time'], y=df_filtered['upwind_efficiency_smooth'], 
                                 mode='lines', name=f"{competitor_id}", 
                                 line=dict(color=color, width=2),
                                 legendgroup=competitor_id, showlegend=False), row=1, col=2)
        
        avg_efficiency = df_filtered['upwind_efficiency_smooth'].mean()
        competitor_efficiencies[competitor_id] = avg_efficiency

    # Add average lines and annotations for both plots
    for col, (averages, title) in enumerate([(competitor_vmg_avg, "VMG"), (competitor_efficiencies, "Efficiency")], start=1):
        sorted_avg = sorted(averages.items(), key=lambda x: x[1], reverse=True)
        y_min = max(0, min(averages.values()) - 1)
        y_max = max(averages.values()) + 1
        y_height = y_max - y_min
        min_separation = 0.1 * y_height
        last_y = y_max

        for i, (competitor_id, avg_value) in enumerate(sorted_avg):
            color = colors[i % len(colors)]
            fig.add_shape(type="line", x0=0, x1=1, xref="x domain", y0=avg_value, y1=avg_value, 
                          line=dict(color=color, width=2, dash="dash"), row=1, col=col)
            
            if abs(avg_value - last_y) < min_separation:
                annotation_y = last_y - min_separation
            else:
                annotation_y = avg_value
            
            fig.add_annotation(text=f"{competitor_id} {avg_value:.2f}", x=1.02, xref="x domain", 
                               y=annotation_y, yref=f"y{col}", showarrow=False, 
                               font=dict(color=color, size=10), xanchor="left", yanchor="middle", row=1, col=col)
            last_y = annotation_y

    fig.update_layout(
        height=700, width=1600,
        title_text="VMG and Upwind Efficiency Comparison",
        legend_title="Competitors",
        hovermode="x unified",
        xaxis=dict(title="Time", tickformat='%H:%M'),
        xaxis2=dict(title="Time", tickformat='%H:%M'),
        yaxis=dict(title="VMG (kts)"),
        yaxis2=dict(title="Upwind Efficiency (%)"),
    )

    fig.update_xaxes(range=[time_start, time_finish])
    fig.update_yaxes(range=[y_min, y_max], row=1, col=1)
    fig.update_yaxes(range=[0, 105], row=1, col=2)

    return fig

In [5]:
def create_race_map_with_plotly_plot(all_gpx_data_with_names, marks, time_start, time_finish):
    # Find the center of the race area
    all_lats = []
    all_lons = []
    for df, _ in all_gpx_data_with_names:
        all_lats.extend(df['latitude'])
        all_lons.extend(df['longitude'])
    
    center_lat = sum(all_lats) / len(all_lats)
    center_lon = sum(all_lons) / len(all_lons)

    # Create the map
    race_map = folium.Map(location=[center_lat, center_lon], zoom_start=14)

    # Define a list of colors for competitors
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

    # Create a list to store all features for TimestampedGeoJson
    all_features = []

    # Add competitor tracks
    for i, (df, file_name) in enumerate(all_gpx_data_with_names):
        color = colors[i % len(colors)]  # Cycle through colors if more competitors than colors
        
        # Create a feature group for this competitor's polyline
        polyline_group = folium.FeatureGroup(name=f'{file_name} Polyline', show=True)
        
        # Add polyline to the feature group
        coordinates = list(zip(df['latitude'], df['longitude']))
        folium.PolyLine(coordinates, color=color, weight=2.5, opacity=1).add_to(polyline_group)
        
        # Add the polyline group to the map
        polyline_group.add_to(race_map)
        
        # Create GeoJSON features for TimestampedGeoJson
        for _, row in df.iterrows():
            feature = {
                'type': 'Feature',
                'geometry': {
                    'type': 'Point',
                    'coordinates': [row['longitude'], row['latitude']]
                },
                'properties': {
                    'time': row['time'].isoformat(),
                    'style': {'color': color},
                    'icon': 'circle',
                    'iconstyle': {
                        'fillColor': color,
                        'fillOpacity': 0.8,
                        'stroke': 'true',
                        'radius': 5
                    },
                    'popup': f"{file_name}<br>Speed: {row['speed_kts']:.2f} kts<br>VMG: {row['VMG']:.2f} kts"
                }
            }
            all_features.append(feature)
        
    # Add a single TimestampedGeoJson layer for all competitors
    timestamped_geojson = TimestampedGeoJson(
        {'type': 'FeatureCollection', 'features': all_features},
        period='PT1S',
        add_last_point=True,
        auto_play=False,
        loop=False,
        max_speed=50,
        loop_button=True,
        date_options='YYYY-MM-DDTHH:mm:ss',
        time_slider_drag_update=True,
        duration='PT200S',
    )
    timestamped_geojson.add_to(race_map)

    # Add the marks to the map
    if marks['Windward']:
        folium.Marker(location=marks['Windward'], popup='Windward Mark', icon=folium.Icon(color='green')).add_to(race_map)
    if marks['Leeward']:
        folium.Marker(location=marks['Leeward'], popup='Leeward Mark', icon=folium.Icon(color='blue')).add_to(race_map)
    if marks['Reach'][0] is not None:
        folium.Marker(location=marks['Reach'], popup='Reach Mark', icon=folium.Icon(color='orange')).add_to(race_map)

    # Generate the combined Plotly figure
    fig_combined = plot_combined_vmg_and_efficiency(all_gpx_data_with_names, time_start, time_finish)
    plot_html_combined = pio.to_html(fig_combined, full_html=False, include_plotlyjs='cdn')
    iframe_combined = IFrame(html=plot_html_combined, width=1650, height=750)
    popup_combined = folium.Popup(iframe_combined, max_width=1700)

    folium.Marker(
        location=[center_lat + 0.008, center_lon + 0.002],
        popup=popup_combined,
        icon=folium.Icon(color='purple', icon='info-sign'),
    ).add_to(race_map)

    # Add layer control to the map
    folium.LayerControl().add_to(race_map)

    return race_map

In [6]:
# File path to store the values
settings_file = '26-05-2024-race1.json'

# Function to load the current values from the JSON file
def load_settings():
    try:
        with open(settings_file, 'r') as f:
            settings = json.load(f)
    except FileNotFoundError:
        settings = {
            'start_time': '2024-05-26T09:45:06Z',
            'finish_time': '2024-05-26T10:29:01Z',
            'upwind_lat': 51.41512328758836,
            'upwind_lon': -0.4701512586325407,
            'gybe_lat': 51.416675,
            'gybe_lon': -0.458546,
            'leeward_lat': 51.422043,
            'leeward_lon': -0.460854
        }
    return settings

# Function to save the current values to the JSON file
def save_settings(settings):
    with open(settings_file, 'w') as f:
        json.dump(settings, f)

# Load the current settings
current_settings = load_settings()

# Define the widgets for input with current values
start_time_input = widgets.Text(description='Start Time:', value=current_settings['start_time'])
finish_time_input = widgets.Text(description='Finish Time:', value=current_settings['finish_time'])
upwind_lat_input = widgets.FloatText(description='Upwind Lat:', value=current_settings['upwind_lat'])
upwind_lon_input = widgets.FloatText(description='Upwind Lon:', value=current_settings['upwind_lon'])
gybe_lat_input = widgets.FloatText(description='Gybe Lat:', value=current_settings['gybe_lat'])
gybe_lon_input = widgets.FloatText(description='Gybe Lon:', value=current_settings['gybe_lon'])
leeward_lat_input = widgets.FloatText(description='Leeward Lat:', value=current_settings['leeward_lat'])
leeward_lon_input = widgets.FloatText(description='Leeward Lon:', value=current_settings['leeward_lon'])

# Display the input widgets
display(start_time_input, finish_time_input, upwind_lat_input, upwind_lon_input, gybe_lat_input, gybe_lon_input, leeward_lat_input, leeward_lon_input)

# Initialize marks globally
marks = {
    'Windward': (None, None),
    'Reach': (None, None),
    'Leeward': (None, None)
}

# Function to get the input values and save them to the JSON file
def get_inputs():
    global marks  # Ensure we use the global marks variable
    settings = {
        'start_time': start_time_input.value,
        'finish_time': finish_time_input.value,
        'upwind_lat': upwind_lat_input.value,
        'upwind_lon': upwind_lon_input.value,
        'gybe_lat': gybe_lat_input.value,
        'gybe_lon': gybe_lon_input.value,
        'leeward_lat': leeward_lat_input.value,
        'leeward_lon': leeward_lon_input.value
    }

    save_settings(settings)
    time_start = pd.to_datetime(settings['start_time']).tz_convert('UTC')
    time_finish = pd.to_datetime(settings['finish_time']).tz_convert('UTC')
    windward_mark = (settings['upwind_lat'], settings['upwind_lon'])
    gybe_mark = (settings['gybe_lat'], settings['gybe_lon'])
    leeward_mark = (settings['leeward_lat'], settings['leeward_lon'])

    # Update the global marks
    marks['Windward'] = windward_mark
    marks['Reach'] = gybe_mark
    marks['Leeward'] = leeward_mark

    return time_start, time_finish, windward_mark, gybe_mark, leeward_mark

# Main execution when apply button is clicked
apply_button = widgets.Button(description='Apply')

def on_apply_button_clicked(b):
    global all_gpx_data_with_names  # Ensure the global variable is used
    time_start, time_finish, windward_mark, gybe_mark, leeward_mark = get_inputs()

    # Reload the GPX files to reset their state
    all_gpx_data_with_names = load_gpx_files(directory_path)

    # Define the marks dictionary
    marks = {
        'Windward': windward_mark,
        'Reach': gybe_mark,
        'Leeward': leeward_mark
    }

    # Process all GPX data
    all_gpx_data_with_names = process_all_data(all_gpx_data_with_names, marks, time_start)

    # Print a message for each processed file
    for df, file_name in all_gpx_data_with_names:
        print(f"Processed {file_name}, DataFrame head:\n{df.head()}")

    # Create and save the race map with the embedded Plotly plots
    race_map_with_plotly_plot = create_race_map_with_plotly_plot(all_gpx_data_with_names, marks, time_start, time_finish)
    race_map_with_plotly_plot.save('race_review.html')

    print("Race review map has been created and saved as 'race_review.html'")

apply_button.on_click(on_apply_button_clicked)
display(apply_button)

Text(value='2024-05-26T09:45:06Z', description='Start Time:')

Text(value='2024-05-26T10:29:01Z', description='Finish Time:')

FloatText(value=51.41512328758836, description='Upwind Lat:')

FloatText(value=-0.4701512586325407, description='Upwind Lon:')

FloatText(value=51.416675, description='Gybe Lat:')

FloatText(value=-0.458546, description='Gybe Lon:')

FloatText(value=51.422043, description='Leeward Lat:')

FloatText(value=-0.460854, description='Leeward Lon:')

Button(description='Apply', style=ButtonStyle())

Found 2 GPX files.
Processing file: 216367_26-05-race 1.gpx
Processed 216367_26-05-race 1.gpx, DataFrame shape: (834, 11)
Processing file: 222201_26-05-race 1.gpx
Processed 222201_26-05-race 1.gpx, DataFrame shape: (710, 11)
Processed 216367_26-05-race 1.gpx, DataFrame head:
                       time   latitude  longitude  speed_kts     heading  \
0 2024-05-26 09:40:27+00:00  51.422402  -0.460203   0.000000    0.000000   
1 2024-05-26 09:40:28+00:00  51.422390  -0.460213   3.002041  207.362656   
2 2024-05-26 09:40:30+00:00  51.422408  -0.460235   2.454557  323.679255   
3 2024-05-26 09:40:33+00:00  51.422476  -0.460259   5.036564  347.600273   
4 2024-05-26 09:40:34+00:00  51.422476  -0.460259   0.000000    0.000000   

   distance  VMG_Windward  VMG_Reach  VMG_Leeward  VMG Sailing_Mode  
0  0.000000           0.0        0.0          0.0  0.0   Not Racing  
1  1.544387           0.0        0.0          0.0  0.0   Not Racing  
2  2.525472           0.0        0.0          0.0  0.0   